In [18]:
import pandas as pd
import numpy as np

In [19]:
df_airbnb = pd.read_csv('airbnb.csv', index_col='id', engine='python', sep =',', encoding="utf-8")
df_airbnb.head()

,latitude,longitude,neighbourhood,bedrooms,cleaning_fee,security_deposit,Free_parking_on_premises,extra_people,Pool,Dryer,...,Crib,Washer,Dishwasher,Suitable_for_events,property_type,Full_kitchen,Internet,Bathtub,Cable_TV,price
id,,,,,,,,,,,,,,,,,,,,,
32046194,-22.98538,-43.19582,0,2.0,73.0,0.0,0,49.0,0,0,...,0,0,0,0,0,0,0,0,0,395.0
2060895,-22.91544,-43.18171,1,1.0,35.0,0.0,0,25.0,0,0,...,0,1,0,1,1,0,1,0,0,84.0
8067388,-22.95565,-43.19536,2,2.0,160.0,0.0,0,90.0,0,0,...,0,1,0,0,0,0,1,0,1,382.0
26081807,-22.98617,-43.22759,3,1.0,120.0,1200.0,0,100.0,0,0,...,0,0,0,0,0,0,0,0,1,151.0
31051808,-22.94991,-43.19289,2,3.0,150.0,0.0,0,70.0,0,0,...,0,1,0,0,0,0,0,0,0,348.0


In [48]:
# Mapa de calor - Concentração de anúncios por bairros
import folium
from folium.plugins import HeatMap
m=folium.Map([-22.92538,-43.29582],zoom_start=12)
HeatMap(df_airbnb[['latitude','longitude']],radius=8,gradient={0.25:'blue',0.5:'purple',0.75:'orange',1.0:'red'}).add_to(m)
display(m)